In [1]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import timm
import torch
import numpy as np

from PIL import Image
from pathlib import Path
from dotenv import load_dotenv
from timm.data.loader import create_loader
from timm.data.dataset import ImageDataset
import torchvision.transforms as transforms

import os
import json

from ViT.tooth_crop_dataset import ToothCropClassDataset
from utils.preprocess import rect_include_another, rotate_bounding_boxes, xyxy_reformat, xyxy2xywh
from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation, bounding_teeth_on_origin, get_all_teeth

load_dotenv()
matplotlib.use('module://matplotlib_inline.backend_inline')

%matplotlib inline


/opt/miniconda3/envs/pt113/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


# Model & env variable loaded

In [2]:
tooth_detect_model = torch.hub.load(r'.\YOLO', 'custom', path=r'.\YOLO\weights\8-bound.pt', source='local')
anomaly_detect_model = torch.hub.load(r'.\YOLO', 'custom', path=r'.\YOLO\weights\anomaly.pt', source='local')

# data_dir = '.' / Path(os.getenv('DATASET_DIR'))
data_dir = Path('..') / 'Datasets' / 'phase-2'


FileNotFoundError: [Errno 2] No such file or directory: '.\\YOLO/hubconf.py'

In [ ]:
image_names = list(data_dir.glob('00008145.jpg'))

image_names = image_names[:20]
image_name = image_names[0]
filename = image_name.stem

im = cv2.imread(image_name)
# Only plt imshow need inverse process
plt.imshow(1 - im)
plt.show()


In [ ]:
results = tooth_detect_model(image_names)
rendered_results = results.render()

plt.imshow(1 - rendered_results[0])
plt.show()


In [ ]:
anomaly_results = anomaly_detect_model(image_names)
rendered_anomaly_results = anomaly_results.render()

plt.imshow(1 - rendered_anomaly_results[0])
plt.show()


In [ ]:
results = tooth_detect_model(image_names)
teeth_region = get_all_teeth(results)
teeth_roi = get_teeth_ROI(results)

teeth_region
teeth_roi


# Save tooth crop image

In [ ]:
temp_dir = '.' / Path(os.getenv('TEMP_DIR')) / 'crop_tooth_image'

missing_tooth = []
for file in temp_dir.glob('*.jpg'):
    os.remove(file)

for filename, region in teeth_region.items():
    for region_name, tooth_region in region.items():
        for tooth_number, data in tooth_region['crop_regions'].items():
            crop_image = data['crop_image']
            is_missing = data['is_missing']

            if is_missing:
                missing_tooth.append((filename, tooth_number))
                continue
            save_filepath = temp_dir / f'{filename} {region_name} {tooth_number}.jpg'

            temp_im = Image.fromarray(crop_image)
            temp_im.save(save_filepath)

# print(save_filepath)
missing_tooth


# Model loading

In [ ]:
model_dir = '.' / Path(os.getenv('ViT_MODEL_DIR'))
model_path = model_dir / 'classifier-6.pt'

vit_model = timm.create_model('swin_base_patch4_window7_224_in22k', num_classes=6)
vit_model.load_state_dict(torch.load(model_path))
vit_model


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 16
num_workers = 0

# Preprocess
transform = transforms.Compose([
    transforms.ToTensor(),
    # (lambda image: padding_to_size(image, 224)),
    transforms.Resize(size=(224, 224)),
    transforms.Normalize(mean=0.5, std=0.5),
])
target_transform = transforms.Compose([
    (lambda y: torch.Tensor(y)),
])
dataset = ImageDataset(temp_dir, transform=transform)

if torch.cuda.is_available():
    dataloader = create_loader(dataset, (3, 224, 224), 4)
else:
    dataloader = create_loader(dataset, (3, 224, 224), 4, use_prefetcher=False)


In [ ]:
size = len(dataloader.dataset)

vit_model.to(device)
vit_model.eval()

threshold = torch.Tensor([0.5, 0.85, 0.5, 0.5, 0.5, 0.5]).to(device)
pred_encodes = []
# target_labels = ['caries', 'endo', 'post', 'crown']
# target_labels = ['R.R', 'caries', 'crown', 'endo', 'filling', 'post']
target_labels = ['caries', 'crown', 'endo', 'filling', 'post']
with torch.no_grad():
    for batch, (X, _) in enumerate(dataloader):
        X = X.to(device)

        # Compute prediction error
        pred = vit_model(X)
        pred = torch.sigmoid(pred)
        pred_encode = pred > threshold
        pred_encodes.append(pred_encode.cpu().numpy())

pred_encodes = np.vstack(pred_encodes)
pred_encodes = pred_encodes[:, 1:]
detected_list = [()] * len(pred_encodes)
for i, pred_encode in enumerate(pred_encodes):
    detected_list[i] = tuple((target_labels[j] for j, checker in enumerate(pred_encode) if checker))

detected_list


In [ ]:
tooth_anomaly_dict = {anomaly_results.files[i][:-4]: {} for i in range(len(anomaly_results))}

for i, detected in enumerate(detected_list):
    current_filename, region_name, tooth_number = dataset.filename(i).split()
    tooth_number = int(tooth_number[:2])

    if tooth_number < 50:
        tooth_anomaly_dict[current_filename][tooth_number] = set(detected)

print(tooth_anomaly_dict)


# YOLO anomaly detect


In [ ]:
iou_threshold = 0.3
region_wisdom_tooth_dict = {
    'upper-left': 18,
    'upper-right': 28,
    'lower-left': 48,
    'lower-right': 38,
}

for i in range(len(anomaly_results)):
    current_filename = anomaly_results.files[i][:-4]
    anomaly_bounds = anomaly_results.xyxy[i]
    for j in range(len(anomaly_bounds)):
        *xyxy, _, cls = anomaly_bounds[j]
        xyxy = list(map(lambda t: t.cpu(), xyxy))

        cls = int(cls.item())
        name = anomaly_results.names[cls]
        # if name in target_labels:
        #     continue

        # for x8 tooth
        if name in ['embedded', 'impacted']:
            min_distance = np.inf
            near_region = ''
            for region_data in teeth_roi['images'][current_filename]:
                region_xyxy = region_data['xyxy']
                region_xywh = xyxy2xywh(np.vstack([region_xyxy]))[0]

                xywh = xyxy2xywh(np.vstack([xyxy]))[0]

                distance = np.linalg.norm(xywh[:2] - region_xywh[:2])
                if distance < min_distance:
                    min_distance = distance
                    near_region = f'{region_data["flag"]}-{region_data["tooth_position"]}'
            if near_region in region_wisdom_tooth_dict:
                tooth_number = region_wisdom_tooth_dict[near_region]
                if tooth_number not in tooth_anomaly_dict[current_filename].keys():
                    tooth_anomaly_dict[current_filename][tooth_number] = {name}
                else:
                    tooth_anomaly_dict[current_filename][tooth_number].add(name)
            continue

        # for normal tooth
        located_regions = {}
        for region_data in teeth_roi['images'][current_filename]:
            region_xyxy = region_data['xyxy']
            if rect_include_another(region_xyxy, xyxy) > iou_threshold:
                located_regions[f'{region_data["flag"]}-{region_data["tooth_position"]}'] = region_data

        for located_region, region_data in located_regions.items():
            region_tooth_data = teeth_region[current_filename][located_region]
            tooth_angle = np.radians(region_tooth_data['angle'])

            offset = region_data['offset']
            region_image_shape = np.array(np.array(region_data['image'].shape)[[1, 0]])

            rotated_xyxy = [xyxy]
            rotated_xyxy = np.array(rotated_xyxy) - np.tile(offset, 2)
            rotated_xyxy = rotate_bounding_boxes(tooth_angle, region_image_shape, rotated_xyxy)
            rotated_xyxy = rotated_xyxy[0].astype(int)

            tooth_number_candidate = []
            for tooth_number, tooth_data in region_tooth_data['crop_regions'].items():
                if tooth_number > 50:
                    continue

                tooth_xyxy = tooth_data['xyxy']
                # print(name)
                # print(rect_include_another(tooth_xyxy, rotated_xyxy))
                tooth_iou = rect_include_another(tooth_xyxy, rotated_xyxy)

                if tooth_iou > 0:
                    tooth_number_candidate.append((tooth_number, tooth_iou))

                # if rect_include_another(tooth_xyxy, rotated_xyxy) > iou_threshold:
                #     tooth_anomaly_dict[current_filename][tooth_number].add(name)
            if tooth_number_candidate:
                tooth_number, tooth_iou = max(tooth_number_candidate, key=lambda t: t[1])
                tooth_anomaly_dict[current_filename][tooth_number].add(name)


for pair in missing_tooth:
    filename, tooth_number = pair
    tooth_anomaly_dict[filename][tooth_number] = {'missing'}

print(tooth_anomaly_dict)


In [ ]:
#
# with open('output.json', 'r') as f:
#     output = json.load(f)
# output[image_name.stem]



